In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd

In [2]:
def getUrl(start, number, board):
    
    for i in range(start,start-number,-1):

        time.sleep(1)

        URL = 'https://www.ptt.cc/bbs/'+board+'/index'+str(i)+'.html'

        # 設定header(cookie)
        set_headers = {'cookie': 'over18=1;'}
        res = requests.get(URL, headers=set_headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        likes = soup.find_all(class_='r-ent')

        for like in likes:
            m = like.find(class_='nrec').text
            if m == '爆':
                m = 100
            elif m.isnumeric():
                m = int(m)
            else:
                m = 0

            if m > 50:
                try:
                    urls.append([m, like.find('a')['href']])
                except:
                    pass
        #         print(like.find(class_='title').text)
        
    return

In [3]:
def fun(path):
    
    # returns the final component of a url
    f_url = os.path.basename(path)
      
    # convert the url into link
    return '<a href="{}">{}</a>'.format(path, f_url)

In [4]:
def getContent(urls):

    i = 0
    data = []

    for url in urls:
        URL = 'https://www.ptt.cc' + url[1]

        set_headers = {'cookie': 'over18=1;'}
        res = requests.get(URL, headers=set_headers)
        soup = BeautifulSoup(res.text, 'html.parser')

        header = soup.find_all('span','article-meta-value')
        author = header[0].text
        board = header[1].text
        title = header[2].text
        date = header[3].text

        main_container = soup.find(id='main-container')
        all_text = main_container.text
    #     pre_text = all_text.split('https://www.ptt.cc/bbs/')[0]
        texts = all_text.split('\n')
        contents = texts[2:]
        content = '\n'.join(contents)

        i+=1
        data.append([url[0], author, board, title, date, fun(URL)])
#         print(f'第{i}篇')
#         print('讚數：'+str(url[0]))
#         print('作者：'+author)
#         print('看板：'+board)
#         print('標題：'+title)
#         print('日期：'+date)
#         print('內文：\n'+content)
#         print('\n============================================================\n')

#         input("下一篇請按Enter")
        
    return data

In [8]:
urls = []

# Gossiping
start = 39200
number = 50
board = 'Gossiping'

getUrl(start, number, board)

# Beauty
# start = 4003
# number = 10
# board = 'Beauty'

# getUrl(start, number, board)

In [9]:
print(f'共有{len(urls)}則貼文\n')

共有36則貼文



In [10]:
data = getContent(urls)
df = pd.DataFrame(data)
df.columns = ['Likes', 'Author', 'Board', 'Title', 'Date', 'Url']
df = df.style.format({'location' : fun})
df

,Likes,Author,Board,Title,Date,Url
0,51,ray198776329 (我真的好帥),Gossiping,[問卦] 什麼叫疫情趨緩所以開國門？哈囉？,Fri Oct 14 02:23:25 2022,M.1665685407.A.8CD.html
1,100,ubcs (覺★青年超冒險蓋),Gossiping,[公告] 跑步哥你又來了 水桶,Fri Oct 14 00:16:36 2022,M.1665677798.A.EDA.html
2,81,mars1985 (╰|∵|╯),Gossiping,[新聞] 認識才7天！北市女大生慘遭網友勒死 嫌,Thu Oct 13 23:24:49 2022,M.1665674692.A.9AD.html
3,77,yesyesyesyes (@[email protected]),Gossiping,[新聞] 「月牙棒阻行器」不怕隨機攻擊 發明人送,Thu Oct 13 22:50:24 2022,M.1665672632.A.C9D.html
4,100,razan (僅供參考),Gossiping,[新聞] 調解破局！ 毆人三重警「賠 10 萬」遭拒,Thu Oct 13 22:23:24 2022,M.1665671006.A.320.html
5,81,jiern (jiern),Gossiping,[問卦] 李秉穎：高端一劑 好像賣1000多元,Thu Oct 13 22:07:00 2022,M.1665670022.A.A76.html
6,54,PBfire (PBfire),Gossiping,[新聞] 台灣人均GDP首度超車日韓 居東亞第一,Thu Oct 13 21:55:51 2022,M.1665669353.A.AFA.html
7,53,SuperSg (○(#‵ ︿′ㄨ)○森77),Gossiping,[問卦] 開價45萬的廖老大頂讓，你會接盤嗎,Thu Oct 13 21:24:27 2022,M.1665667472.A.6F1.html
8,67,neo19 ((づ′・ω・）づ),Gossiping,[爆卦] 索尼中國又乳華了,Thu Oct 13 21:30:06 2022,M.1665667809.A.38F.html
9,83,wade2012 (臭鼻),Gossiping,[問卦] 陽台撿到鳥怎麼辦,Thu Oct 13 21:01:18 2022,M.1665666080.A.086.html
